**Mengimpor Modul untuk Pemrosesan Data, Permintaan HTTP, dan TensorFlow**

Kode ini mengimpor beberapa modul Python yang diperlukan untuk berbagai tugas, termasuk `base64` dan `json` untuk encoding dan decoding data, `random` untuk operasi acak, `pandas` untuk manipulasi dan analisis data, `requests` untuk membuat permintaan HTTP, dan `tensorflow` untuk operasi pembelajaran mesin dan pemrosesan data tensor. Fungsinya adalah untuk menyediakan semua alat yang diperlukan untuk menangani data, melakukan permintaan jaringan, dan menerapkan model pembelajaran mesin dalam skrip atau aplikasi yang sedang dikembangkan.

In [253]:
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

**Membaca Dataset Stroke dari File CSV**

Kode ini menggunakan Pandas untuk membaca file CSV yang berisi dataset kesehatan terkait stroke dari path yang ditentukan dan menyimpannya dalam variabel `data_csv`.

In [254]:
data_csv = pd.read_csv(r"C:\Users\ANANG\ML_Ops\Yafi\data\healthcare-dataset-stroke-data.csv")

**Mengambil Sampel Acak dari Dataset dan Membuat Dictionary Fitur**

Kode ini mengambil nama kolom dari dataset, memilih baris acak dari dataset, dan menyimpan nilai-nilai fitur dari baris tersebut. Kemudian, kode membuat dictionary `inputs` yang memetakan setiap nama kolom ke nilai fiturnya yang sesuai, menyediakan representasi fitur dari sampel acak dari dataset.

In [255]:
columns = data_csv.columns.values
rand = random.randint(0, len(data_csv) - 1)

features = data_csv.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'id': 32150,
 'gender': 'Female',
 'age': 56.0,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'Yes',
 'work_type': 'Self-employed',
 'Residence_type': 'Rural',
 'avg_glucose_level': 94.71,
 'bmi': 29.6,
 'smoking_status': 'smokes',
 'stroke': 0}

**Mendefinisikan Fungsi untuk Membuat Fitur TensorFlow dari Nilai Data**

Kode ini mendefinisikan tiga fungsi: `string_feature`, `float_feature`, dan `int_feature`, yang masing-masing mengubah nilai data menjadi objek `tf.train.Feature` yang sesuai. Fungsi-fungsi ini digunakan untuk mengkonversi nilai string, float, dan integer menjadi format yang dapat digunakan dalam protokol buffer TensorFlow untuk menyimpan data fitur.

In [256]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

**Menyiapkan JSON untuk Permintaan Inferensi TensorFlow Serving**

Kode ini mendefinisikan fungsi `prepare_json` yang mengonversi dictionary `inputs` menjadi format JSON yang dapat digunakan untuk permintaan inferensi dengan TensorFlow Serving. Fungsi ini mengonversi nilai-nilai dalam `inputs` menjadi fitur TensorFlow yang sesuai, membungkusnya dalam objek `tf.train.Example`, dan kemudian mengenkodekannya ke dalam format base64 sebelum mengemasnya menjadi JSON.

In [257]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

**Membuat Prediksi Penyakit Stroke Menggunakan TensorFlow Serving**

Kode ini mendefinisikan fungsi `make_predictions` yang mempersiapkan data input menjadi JSON menggunakan `prepare_json`, mengirim permintaan POST ke endpoint model TensorFlow Serving, dan menerima respons prediksi. Berdasarkan hasil prediksi, fungsi ini mengembalikan diagnosis apakah seseorang "Tidak Mengalami Penyakit Stroke" atau "Mengalami Penyakit Stroke".

In [258]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://muhammadyafi079-pipeline-production.up.railway.app/v1/models/stroke-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.5:
        return "Tidak Mengalami Penyakit Stroke"
    else:
        return "Mengalami Penyakit Stroke"

**Mencetak Hasil Prediksi dan Fitur Input untuk Diagnosis Stroke**

Kode ini memanggil fungsi `make_predictions` dengan `inputs` sebagai argumen untuk mendapatkan hasil prediksi penyakit stroke. Kemudian, hasil prediksi dan fitur input yang digunakan untuk prediksi dicetak ke layar.

In [259]:
hasil_prediksi = make_predictions(inputs)
print(f'Hasil Prediksi: {hasil_prediksi}')
print('Fitur Input:', inputs)

Hasil Prediksi: Tidak Mengalami Penyakit Stroke
Fitur Input: {'id': 32150, 'gender': 'Female', 'age': 56.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Self-employed', 'Residence_type': 'Rural', 'avg_glucose_level': 94.71, 'bmi': 29.6, 'smoking_status': 'smokes', 'stroke': 0}
